# Title

This Notebook...

## Initialization

### Imports

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(message)s')

import numpy
import matplotlib.pyplot

import sys
sys.path.append("../")
import congestionfinder
import congestionfinder.bpsdetector
import congestionfinder.road
import congestionfinder.detection
import congestionfinder.speedflow
import congestionfinder.congestion
import congestionfinder.boundaryscanner

## Classes

In [ ]:
bpsDetector = congestionfinder.bpsdetector.BPSDetector("00D00C03405B18200005")
print(bpsDetector)

In [ ]:
road = congestionfinder.road.Road(12)
road.addBPSDetector(bpsDetector)
print(road)
road.indexDetectorSpaces()
print(road.getSpaceToSpaceIndex())

In [ ]:
detection = congestionfinder.detection.Detection("00D00C03405B18200005", 1234, 1440, 120, 17)
print(detection)

## Roads

In [ ]:
roads = congestionfinder.road.parseBPSCodesToRoads(["10D00204D037D007000B", "10D002051800D0070013", "10D002057000D007000F"])
for value in roads.values():
    value.indexDetectorSpaces

In [ ]:
for value in sorted(roads.values(), key = lambda x:len(x.getBPSDetectors())):
    print(value)

## Speed and Flows

In [ ]:
detections = []
detections.append(congestionfinder.detection.Detection("10D00204D037D007000B", 30855, 1, 80, 17))
detections.append(congestionfinder.detection.Detection("10D00204D037D007000B", 30855, 2, 70, 16))
detections.append(congestionfinder.detection.Detection("10D00204D037D007000B", 30855, 3, 75, 17))
detections.append(congestionfinder.detection.Detection("10D002051800D0070013", 32600, 1, 40, 10))
detections.append(congestionfinder.detection.Detection("10D002051800D0070013", 32600, 2, 55, 18))
detections.append(congestionfinder.detection.Detection("10D002051800D0070013", 32600, 3, 35, 12))
detections.append(congestionfinder.detection.Detection("10D002057000D007000F", 34800, 1, 70, 19))
detections.append(congestionfinder.detection.Detection("10D002057000D007000F", 34800, 2, 85, 14))
detections.append(congestionfinder.detection.Detection("10D002057000D007000F", 34800, 3, 100, 19))

In [ ]:
speeds, flows, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex = congestionfinder.speedflow.parseDetectionsToSpeedsAndFlows(detections, roads[2])

In [ ]:
print(speeds.shape)
print(flows.shape)
print(minSpaceIndex)
print(maxSpaceIndex)
print(minTimeIndex)
print(maxTimeIndex)
matplotlib.pyplot.imshow(speeds, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flows, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsHighFlow, flowsHighFlow, maskHighFlow = congestionfinder.speedflow.removeLowFlowTimes(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsHighFlow.shape)
print(flowsHighFlow.shape)
print(sum(maskHighFlow) / (sum(maskHighFlow) + sum(~maskHighFlow)))
matplotlib.pyplot.imshow(speedsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsWorkingDetectors, flowsWorkingDetectors, maskWorkingDetectors = congestionfinder.speedflow.removeMissingDetectors(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsWorkingDetectors.shape)
print(flowsWorkingDetectors.shape)
print(sum(maskWorkingDetectors) / (sum(maskWorkingDetectors) + sum(~maskWorkingDetectors)))
matplotlib.pyplot.imshow(speedsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion

In [ ]:
congestions = congestionfinder.congestion.parseSpeedFlowsToCongestions(speedsWorkingDetectors, flowsWorkingDetectors)

In [ ]:
print(congestions.shape)
matplotlib.pyplot.imshow(congestions, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
congestionsWithoutMissingValues = congestionfinder.congestion.interpolateMissingValues(congestions)

In [ ]:
print(congestionsWithoutMissingValues.shape)
matplotlib.pyplot.imshow(congestionsWithoutMissingValues, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
congestionsSmoothed = congestionfinder.congestion.applySmoothingFilter(congestionsWithoutMissingValues)

In [ ]:
print(congestionsSmoothed.shape)
matplotlib.pyplot.imshow(congestionsSmoothed, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion Filter

In [ ]:
testDataArray = numpy.random.rand(16,9)
print(testDataArray >= 0.9)

In [ ]:
congestionfinder.boundaryscanner.scanForBoundaries(testDataArray, threshold = 0.1, depth = 0)

In [ ]:
testBoundariesList = congestionfinder.boundaryscanner.recursiveScanForBoundaries(testDataArray, threshold = 0.1)

In [ ]:
testBoundariesList

In [ ]:
congestionfinder.boundaryscanner.plotCongestionsWithBoundaries(testDataArray, testBoundariesList)
congestionfinder.boundaryscanner.plotCongestionsWithBoundaries(testDataArray < 0.1, testBoundariesList)

In [ ]:
congestionBoundariesList = congestionfinder.boundaryscanner.recursiveScanForBoundaries(congestionsSmoothed)

In [ ]:
print(congestionBoundariesList)

In [ ]:
congestionBoundariesListFiltered = congestionfinder.boundaryscanner.filterLargeCongestions(congestionBoundariesList)

In [ ]:
print(congestionBoundariesListFiltered)

In [ ]:
congestionBoundariesListWithMargins = congestionfinder.boundaryscanner.addMargins(congestionBoundariesListFiltered, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex)

In [ ]:
print(congestionBoundariesListWithMargins)

In [ ]:
congestionfinder.boundaryscanner.plotCongestionsWithBoundaries(congestionsSmoothed, congestionBoundariesListWithMargins)
congestionfinder.boundaryscanner.plotCongestionsWithBoundaries(congestionsSmoothed < 1, congestionBoundariesList)

In [ ]:
congestionfinder.boundaryscanner.plotCongestionsWithBoundaries(speeds, congestionBoundariesListWithMargins)

End of Notebook